In [1]:
import pandas as pd
import stanza
import requests
from bs4 import BeautifulSoup
import time

#Stanza pipeline suomen kieltä varten
nlp = stanza.Pipeline('fi', processors='tokenize,pos,lemma', use_gpu=False)

#Suomenkielisten stop-sanojen lista
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-fi/master/stopwords-fi.txt"
response = requests.get(url)
#Muodostetaan stop sanoista lista, jossa jokainen sana on oma alkionsa
finnish_stop_words = response.text.splitlines()
suodatus = {'ei', 'eikä', 'mutta', 'vaan', 'vaikka', 'jos', 'kuin'} #näitä ei poisteta
custom_stop_words = [word for word in finnish_stop_words if word not in suodatus]
custom_stop_words.append('placeholder') #tyhjien otsikoiden tilalle laitettu teksti
print(f"Alkuperäinen stop-sanojen määrä: {len(finnish_stop_words)}")
print(f"Karsittu stop-sanojen määrä: {len(custom_stop_words)}")
print("Esimerkkejä:", custom_stop_words[10:20])

#Funktio tekstin lemmatisointia varten
def lemmatize_text(text):
    doc = nlp(text)
    lemmatized_words = [word.lemma for sent in doc.sentences for word in sent.words]
    return ' '.join(lemmatized_words)

# Aineisto
df = pd.read_csv('arvostelut.csv', delimiter=';', quoting=0)
print(df.head())
#Lemmatisoinnin sovellus
# Syöte X, data jonka avulla malli oppii. Kohde y, se mitä mallin tulee oppia ennustamaan.
df['lemmatized_text'] = df['teksti'].apply(lemmatize_text)
df['lemmatized_title'] = df['otsikko'].apply(lemmatize_text)
X = df['lemmatized_title'] + df['lemmatized_text']
#X = df['teksti']
y = df['tunne']

# train_test_split sekoittaa datan ja jakaa sen neljään osaan: X_train ja y_train opetusdata, X_test ja y_test testidata
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=77, stratify=y
)

# Pipeline: TF-IDF + Logistic Regression
from sklearn.pipeline import Pipeline #ketjuttaa datan muokkausvaiheet ja malli yhteen pakettiin
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=custom_stop_words)), #vastaanottaa tekstin ja muuttaa sen numeroiksi TF-IDF:n avulla
    ('clf', LogisticRegression(class_weight='balanced')), #Luokittelija, ottaa vastaan TfidVectorizer:n tuottaman numerodatan ja tekee ennusteita sen perusteella. 
    #('clf', MultinomialNB()),
    #('clf', LinearSVC(class_weight='balanced', dual="auto")),
])


text_clf.fit(X_train, y_train) #.fit() komento syöttää opetusdatan X_train ja y_train putken läpi, TfidVectorizer analysoi X_trainin sanat ja luo sanaston sekä laskee TF-IDF arvot.

#Numerodata ja vastaukset y_train syötetään mallille joka oppii mitkä sanat liittyvät mihinkin tunteeseen
from sklearn.metrics import classification_report, accuracy_score
predictions = text_clf.predict(X_test) #Ennusteet testidatalla
#Tulostetaan raportti
print(f"Accuracy: {accuracy_score(y_test, predictions)}")
print(classification_report(y_test, predictions))


2025-08-25 20:05:50 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-08-25 20:05:50 INFO: Downloaded file to C:\Users\huvif\stanza_resources\resources.json
2025-08-25 20:05:50 WARNING: Language fi package default expects mwt, which has been added
2025-08-25 20:05:50 INFO: Loading these models for language: fi (Finnish):
| Processor | Package      |
----------------------------
| tokenize  | tdt          |
| mwt       | tdt          |
| pos       | tdt_charlm   |
| lemma     | tdt_nocharlm |

2025-08-25 20:05:50 INFO: Using device: cpu
2025-08-25 20:05:50 INFO: Loading: tokenize
2025-08-25 20:05:51 INFO: Loading: mwt
2025-08-25 20:05:51 INFO: Loading: pos
2025-08-25 20:05:51 INFO: Loading: lemma
2025-08-25 20:05:52 INFO: Done loading processors!


Alkuperäinen stop-sanojen määrä: 847
Karsittu stop-sanojen määrä: 841
Esimerkkejä: ['aikovat', 'aina', 'ainakaan', 'ainakin', 'ainoa', 'ainoat', 'aiomme', 'aion', 'aiotte', 'aist']
       otsikko                                             teksti   tahdet  \
0  placeholder  Ostin hyvästä tarjouksesta pesukoneen asennuks...  5.touko   
1  placeholder  Ensimmäiset viikot pyykätty ja tuntuisi olevan...  4.touko   
2  placeholder  Onhan näissä koneissa eroa kun ostaa 300€ kone...  5.touko   
3  placeholder  Vasta kaksi kertaa käytetty: mix -ohjelma ja l...  4.touko   
4  placeholder  Hommattiin pari viikkoa sitten ja oikeastaan s...  5.touko   

                                                 url         tunne  
0  https://www.gigantti.fi/product/kodinkoneet/py...  positiivinen  
1  https://www.gigantti.fi/product/kodinkoneet/py...  positiivinen  
2  https://www.gigantti.fi/product/kodinkoneet/py...  positiivinen  
3  https://www.gigantti.fi/product/kodinkoneet/py...  positiivinen  
4  ht